In [127]:
class Room:
    def __init__(self, schedule, room_name):
        self.name = room_name
        self.schedule = [x for x in schedule]

    def __str__(self):
        return '{} {}'.format(self.schedule, self.name)
    
    def __repr__(self):
        return '{} {}'.format(self.schedule, self.name)
    
    def book_hour(self, hour):
        if self.schedule[hour] == '.':
            self.schedule[hour] = 'X'
        else:
            raise ValueError('Hour {} in room {} is already booked.'.format(hour, self.name))

    # first available hour in a room
    def fah_room(self):
        return self.schedule.index('.')

    def is_available(self, hour):
        if self.schedule[hour] == '.':
            return True
        else:
            return False


class City:
    def __init__(self, city_name, n_rooms):
        self.name = city_name
        self.number_of_rooms = n_rooms
        self.rooms = []

    def __repr__(self):
        return f'{self.name}, {self.number_of_rooms}: ' + ' '.join(str(x) for x in self.rooms)

    def add_room(self, room):
        if len(self.rooms) < self.number_of_rooms:
            if isinstance(room, Room):
                self.rooms.append(room)
            else:
                raise TypeError('Expected Room object, got {} instead.'.format(type(room)))
        else:
            raise ValueError('Maximum number of {} rooms is already reached.'.format(self.number_of_rooms))

    # first available hour in a city
    def fah_city(self):
        return min([room.fah_room() for room in self.rooms])

    def has_available_room(self, hour):
        not_found = True
        i = 0
        while not_found and i < len(self.rooms):
            if self.rooms[i].is_available(hour):
                not_found = False
                return self.rooms[i].name
            i+=1
        return None
    
class CallRequest:
    def __init__(self, n_cities):
        self.number_of_cities = n_cities
        self.cities = []
    
    def add_city(self, city):
        if len(self.cities) < self.number_of_cities:
            if isinstance(city, City):
                self.cities.append(city)
            else:
                raise TypeError('Expected City object, got {} instead.'.format(type(city)))
        else:
            raise ValueError('Maximum number of {} cities is already reached.'.format(self.number_of_cities))

    def get_starting_hour(self):
        return max([city.fah_city() for city in self.cities])

    def make_booking(self):
        global ROOMS
        booked = False
        hour = self.get_starting_hour()
        while hour < 24 and not booked:
            booked = self.check_all_cities(hour)
            if booked:
                for room in booked:
                    ROOMS[room].book_hour(hour)
                print('Yes ' + ' '.join(str(x) for x in booked))
            hour+=1
        if not booked:
            print('No')
        

    def check_all_cities(self, hour):
        result = []
        failed = False
        i = 0
        while i < len(self.cities) and not failed:
            room = self.cities[i].has_available_room(hour)
            if not room:
                failed = True
            else:
                result.append(room)
            i+=1
        return result if len(result) == self.number_of_cities else None

In [128]:
f = open('input.txt', 'r')

CITIES = []
ROOMS = {}
N_CITIES = int(f.readline())
for i in range(N_CITIES):
    args = f.readline().split()
    CITIES.append(City(args[0],int(args[1])))
    for j in range(CITIES[-1].number_of_rooms):
        args = f.readline().split()
        ROOMS[args[1]] = Room(args[0], args[1])
        CITIES[-1].add_room(ROOMS[args[1]])
        
CITIES = {x.name: x for x in CITIES}

N_REQUESTS = int(f.readline())
for i in range(N_REQUESTS):
    args = f.readline().split()
    request = CallRequest(int(args[0]))
    for k in args[1:]:
        request.add_city(CITIES[k])
    request.make_booking()

No
Yes Kvartal Toloka
Yes Toloka Mitte
Yes Kvartal Lustgarten
